In [73]:
import urllib.request
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import unicodedata

In [41]:
driver = webdriver.Chrome('C:/Users/LG/Desktop/21winter/YBIGTA/edu_session/0202 크롤링/chromedriver.exe')
driver2 = webdriver.Chrome('C:/Users/LG/Desktop/21winter/YBIGTA/edu_session/0202 크롤링/chromedriver.exe')

In [42]:
# 로그인
driver.get('https://www.instagram.com/explore/')
driver2.get('https://www.instagram.com/explore/')
time.sleep(1)
# 인스타 ID와 PW
driver.find_element_by_name('username').send_keys('')
driver.find_element_by_name('password').send_keys('')
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button').click()
driver2.find_element_by_name('username').send_keys('')
driver2.find_element_by_name('password').send_keys('')
driver2.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button').click()

In [43]:
how_many_scroll = 3

# how_many_scroll만큼 맨 밑으로 스크롤하고 맨 위로 돌아오기
for _ in range(how_many_scroll):
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(3)
driver.execute_script('window.scrollTo(0,0);')

# 첫 번째 게시물
driver.find_element_by_class_name('_9AhH0').click()

In [63]:
image_name_list = []
image_url_list = []
contents_list = []
location_list = []
hashtags_list = []

In [70]:
# 이미지를 따로 보관할 'img'라는 디렉토리를 미리 생성
how_many_crawl = 5
n = 0

for i in range(how_many_crawl):
    page_url = driver.current_url
    driver2.get(page_url)
    time.sleep(1)
    
    html = driver2.page_source
    soup = bs(html, 'lxml')
    
    img_url = soup.select_one('.KL4Bh').img['src']
    image_name = 'test_' + str(n) + '.jpg'
    with urllib.request.urlopen(img_url) as f:
        with open('./img/' + 'test_' + str(n) + '.jpg', 'wb') as h:
            img = f.read()
            h.write(img)

    location = driver2.find_elements_by_class_name('O4GlU')
    if len(location) == 0:
        location = ''
    else:
        location = location[0].text.replace(',',' ')
        
    contents = driver2.find_elements_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span')
    if len(contents) == 0:
        contents = ''
    else:
        contents = contents[0].text.split('#')[0]
    
    # 답글 열기 - test
    replies = driver2.find_elements_by_class_name('EizgU')
    for reply in replies:
        reply.click()
    time.sleep(1)
    
    hashtags = ''
    tag_list = driver2.find_elements_by_class_name('xil3i')
    for k in range(len(tag_list)):
        hashtags += tag_list[k].text
        
    image_url_list.append(img_url)
    image_name_list.append(image_name)
    location_list.append(location)
    hashtags_list.append(hashtags)
    contents_list.append(contents)
    
    # 다음 게시물
    driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
    time.sleep(1)
    n += 1
     

In [75]:
# 자음 모음 분리 현상 해결
for i in range(len(contents_list)):
    contents_list[i] = unicodedata.normalize('NFC', contents_list[i])
for i in range(len(location_list)):
    location_list[i] = unicodedata.normalize('NFC', location_list[i])    
for i in range(len(hashtags_list)):
    hashtags_list[i] = unicodedata.normalize('NFC', hashtags_list[i])

In [76]:
dictionary = {'image_name':image_name_list,
             'image_url':image_url_list,
             'contents':contents_list,
             'location':location_list,
             'hashtags':hashtags_list}
df = pd.DataFrame(dictionary)
df

,image_name,image_url,contents,location,hashtags
0,test_0.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,탈출의 기록_20210213\n.\n연휴내내 집콕하다가 3일만에 엄빠랑 탈출😎\n날...,에브리선데이커피 Everysundaycoffee,#세종#고복저수지카페#마카다미아쿠키#베리베리에이드#바깥생활끝#다시집콕#다시넷플릭스로...
1,test_1.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,초코송이와 젤리로 윌리엄의 뒤태 만드는 법 😂\n.\n.\n더 많은 꿀잼 자료는 👉...,,#초코송이#젤리#윌리엄#뒤태#ㅋㅋㅋㅋㅋㅋ#졸귀#핵귀#귀요미#짱귀#유머짤#팔로워환영#...
2,test_2.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,아나 치킨먹다 놀라 기절할뻔;;\n혓바닥 어쩔거얔ㅋㅋㅋ 제발 치워줘..\n_____...,,#유머스타그램#개그#웃긴짤#짤방#재미#재미스타그램#웃긴#요즘#공감#심심#꿀잼#핵잼#...
3,test_3.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,,,#에즈금융서비스#선팔#맞팔#선팔맞팔#선팔하면맞팔#좋아요#다이렉트#취업#취미#여유#채...
4,test_4.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,이사배 금손 인증~!!!🥰🥰🥰\n.\n.\n,홍대입구역,#좋아요#좋반#좋튀#좋테#예능#꿀잼#개꿀잼#대유잼#병맛#힐링#선팔#맞팔#선팔하면맞팔...


In [78]:
df.to_csv("data_list.csv", index=True, encoding='utf-8-sig')